# 2. Scraping
Scrape the [Top 10 Corporations by Year data](https://en.wikipedia.org/wiki/List_of_public_corporations_by_market_capitalization) from Wikipedia.

In [53]:
import bs4
import requests, re

In [16]:
url = 'http://en.wikipedia.org/wiki/List_of_public_corporations_by_market_capitalization'
res = requests.get(url)
soup = bs4.BeautifulSoup(res.text, 'html5lib')

In [46]:
tables = soup.find_all('table', {'class': 'wikitable'})
len(tables)

30

Filter out tables that aren't what we want. We'll check for the presence of a column called "First Quarter".

In [47]:
def is_data_table(table):
    headers = table.find_all('th')
    header_text = ','.join([header.text for header in headers])
    if 'first quarter' not in header_text.lower():
        return False
    return True

tables = list(filter(is_data_table, tables))
len(tables)

11

### Get the company names out of the table cells.

This function will work on the tables from 2008-2018

In [80]:
def extract_text_from_td(td):
    text = td.text
    regex = re.compile('(.*)\n.*')
    match = regex.search(text)
    if match is not None:
        return match.group(1)

extract_text_from_td(tables[0].find_all('td')[2])

'Apple Inc.'

In [105]:
company_names = set()
for table in tables:
    tds = table.find_all('td')
    text = map(extract_text_from_td, tds)
    text = filter(lambda x: x is not None and len(x) > 0, text)
    company_names |= set(text)

print(company_names)

{'Alphabet Inc.', 'China Mobile', 'HSBC', 'Walmart', 'Nestlé', 'Wells Fargo', 'IBM', 'Royal Dutch Shell', 'Industrial and Commercial Bank of China', 'AT&T', 'Petrobras', 'PetroChina', 'Wal-Mart', 'Tencent', 'Exxon Mobil', 'Alibaba Group', 'Facebook', 'General Electric', 'Google', 'Petro China', 'Procter & Gamble', 'Chevron Corporation', 'ICBC', 'Microsoft', 'BHP Billiton', 'Alphabet', 'Novartis', 'Berkshire Hathaway', 'Johnson & Johnson', 'Hoffmann-La Roche', 'Amazon', 'Samsung Electronics', 'China Construction Bank', 'Gazprom', 'Amazon.com', 'ExxonMobil', 'JPMorgan Chase', 'Apple Inc.'}


### TODO: Get the companies from the tables before 2008